In [ ]:
import cv2, os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras, sys, time, warnings
from keras.models import *
from keras.layers import *
from keras import optimizers

In [ ]:
data = 'idd-lite_1/idd20k_lite/'

img_train = data + 'leftImg8bit/train/'
seg_train = data + 'gtFine/train/'

img_val = data + 'leftImg8bit/val/'
seg_val = data + 'gtFine/val/'

In [ ]:
mi, ma = 0, 6
n_classes = ma - mi + 1

In [ ]:
def getImageArr(path, width, height):
    img = cv2.imread(path, 1)
    img = np.float32(cv2.resize(img, (width, height))) / 127.5 - 1
    return img

def getSegmentationArr(path, nClasses, width, height):
    seg_labels = np.zeros((height, width, nClasses))
    img = cv2.imread(path, 1)
    img = cv2.resize(img, (width, height))
    img = img[:, :, 0]

    for c in range(nClasses):
        seg_labels[:, :, c] = (img == c ).astype(int)

    return seg_labels

In [ ]:
input_height, input_width = 224, 224
output_height, output_width = 224, 224

In [ ]:
X_train, y_train = [], []
X_val, y_val = [], []

In [ ]:
train_img = os.listdir(img_train)
train_img.sort()
train_seg = os.listdir(seg_train)
train_seg.sort()

val_img = os.listdir(img_val)
val_img.sort()
val_seg = os.listdir(seg_val)
val_seg.sort()

In [ ]:
print(train_img[:5])
print(train_seg[:5])
print(val_img[:5])
print(val_seg[:5])

In [ ]:
for im, seg in zip(train_img, train_seg):
    X_train.append(getImageArr(img_train + im, input_width, input_height))
    y_train.append(getSegmentationArr(seg_train + seg, n_classes, output_width, output_height))
    
for im, seg in zip(val_img, val_seg):
    X_val.append(getImageArr(img_val + im, input_width, input_height))
    y_val.append(getSegmentationArr(seg_val + seg, n_classes, output_width, output_height))

In [ ]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val),np.array(y_val)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

In [ ]:
VGG_Weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
def FCN8(nClasses, input_width = 224, input_height = 224):
    assert input_width % 32 == 0
    assert input_height % 32 == 0
    IMAGE_ORDERING = 'channels_last'
    
    img_input = Input(shape = (input_height, input_width, 3))
    
    x = Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv1', data_format = IMAGE_ORDERING )(img_input)
    x = Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv2', data_format = IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides = (2, 2), name = 'block1_pool', data_format = IMAGE_ORDERING )(x)
    f1 = x
    
    x = Conv2D(128, (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv1', data_format = IMAGE_ORDERING )(x)
    x = Conv2D(128, (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv2', data_format = IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides = (2, 2), name = 'block2_pool', data_format = IMAGE_ORDERING )(x)
    f2 = x
    
    x = Conv2D(256, (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv1', data_format = IMAGE_ORDERING )(x)
    x = Conv2D(256, (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv2', data_format = IMAGE_ORDERING )(x)
    x = Conv2D(256, (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv3', data_format = IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides = (2, 2), name = 'block3_pool', data_format = IMAGE_ORDERING )(x)
    pool3 = x
    
    x = Conv2D(512, (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv1', data_format = IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv2', data_format = IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv3', data_format = IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides = (2, 2), name = 'block4_pool', data_format = IMAGE_ORDERING )(x)
    pool4 = x
    
    x = Conv2D(512, (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv1', data_format = IMAGE_ORDERING )(pool4)
    x = Conv2D(512, (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv2', data_format = IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv3', data_format = IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides = (2, 2), name = 'block5_pool', data_format = IMAGE_ORDERING )(x)
    pool5 = x
    
    vgg = Model(img_input, pool5)
    vgg.load_weights(VGG_Weights_path)
    
    n = 4096
    o = (Conv2D(n, (7, 7), activation = 'relu', padding = 'same', name = 'conv6', data_format = IMAGE_ORDERING))(pool5)
    conv7 = (Conv2D(n, (1, 1), activation = 'relu', padding = 'same', name = 'conv7', data_format = IMAGE_ORDERING))(o)
    
    conv7_4 = Conv2DTranspose(nClasses, kernel_size = (4, 4) ,strides = (4, 4), use_bias = False, data_format = IMAGE_ORDERING )(conv7)
    
    pool411 = (Conv2D(nClasses, (1, 1), activation = 'relu', padding = 'same', name = 'pool4_11', data_format = IMAGE_ORDERING))(pool4)
    pool411_2 = (Conv2DTranspose(nClasses, kernel_size = (2, 2), strides = (2, 2), use_bias = False, data_format = IMAGE_ORDERING ))(pool411)
    
    pool311 = (Conv2D(nClasses, (1, 1), activation = 'relu', padding = 'same', name = 'pool3_11', data_format = IMAGE_ORDERING))(pool3)
        
    o = Add(name = 'add')([pool411_2, pool311, conv7_4])
    o = Conv2DTranspose(nClasses, kernel_size = (8, 8), strides = (8, 8), use_bias = False, data_format = IMAGE_ORDERING )(o)
    o = (Activation('softmax'))(o)
    
    model = Model(img_input, o)

    return model

In [ ]:
model = FCN8(nClasses = n_classes, input_width  = 224, input_height = 224)
model.summary()

In [ ]:
sgd = optimizers.SGD(lr = 0.01, decay = 5 ** (-4), momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy',
              optimizer = sgd,
              metrics = ['accuracy'])

In [ ]:
hist1 = model.fit(X_train, y_train,
                  validation_data = (X_val, y_val),
                  batch_size = 32, epochs = 200, verbose = 1)

In [ ]:
for key in ['loss', 'val_loss']:
    plt.plot(hist1.history[key], label = key)
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(X_val)
y_predi = np.argmax(y_pred, axis=3)
y_testi = np.argmax(y_val, axis=3)
print(y_testi.shape,y_predi.shape)

In [ ]:
def IoU(yi, y_predi):
    IoUs = []
    Nclass = 7
    for c in range(Nclass):
        TP = np.sum((yi == c) & (y_predi == c))
        FP = np.sum((yi != c) & (y_predi == c))
        FN = np.sum((yi == c) & (y_predi != c)) 
        IoU = TP/float(TP + FP + FN)
        print('class {:02.0f}: #TP={:6.0f}, #FP={:6.0f}, #FN={:5.0f}, IoU={:4.3f}'.format(c, TP, FP, FN, IoU))
        IoUs.append(IoU)
    mIoU = np.mean(IoUs)
    print('Mean IoU: {:4.3f}'.format(mIoU))

In [ ]:
IoU(y_testi,y_predi)

In [ ]:
model.save('fcn.h5')